## Formatted Zone

In [6]:
# Importing useful dependencies
import boto3

In [7]:
# Setup S3 client for MinIO (MinIO implements Amazon S3 API)
s3 = boto3.client(
    "s3",
    endpoint_url="http://127.0.0.1:9000", # MinIO API endpoint
    aws_access_key_id="minioadmin", # User name
    aws_secret_access_key="minioadmin", # Password
)

In [8]:
# This function copies all objects from a bucket (under a prefix) into another bucket, creating the destination bucket if it doesn’t exist.
def replicate_bucket(src_bucket, dest_bucket, src_prefix=""):
    # Create destination bucket if it doesn't exist
    existing = s3.list_buckets()
    if dest_bucket not in [b["Name"] for b in existing["Buckets"]]:
        s3.create_bucket(Bucket=dest_bucket)

    paginator = s3.get_paginator("list_objects_v2") # It returns objects in pages and not all at once.
    for page in paginator.paginate(Bucket=src_bucket, Prefix=src_prefix):
        for obj in page.get("Contents", []):
            
            key = obj["Key"]
            
            if obj['Size'] == 0 and key.endswith("/"): # skip the folder itself
                continue

            # Remove the prefix part from the key
            new_key = key[len(src_prefix):]

            # Copy object without top-level folder
            copy_source = {"Bucket": src_bucket, "Key": key}
            s3.copy_object(Bucket=dest_bucket, Key=new_key, CopySource=copy_source)

            print(f"Copied: {key} -> {new_key}")

In [9]:
# Replicate files from Landing Zone to Formatted Zone
replicate_bucket(src_bucket = "landing-zone", src_prefix="persistent-landing", dest_bucket = "formatted-zone")

Copied: persistent-landing/images/image_1762966866790.jpg -> /images/image_1762966866790.jpg
Copied: persistent-landing/images/image_1762966866959.jpg -> /images/image_1762966866959.jpg
Copied: persistent-landing/images/image_1762966867034.jpg -> /images/image_1762966867034.jpg
Copied: persistent-landing/images/image_1762966867095.jpg -> /images/image_1762966867095.jpg
Copied: persistent-landing/images/image_1762966867157.jpg -> /images/image_1762966867157.jpg
Copied: persistent-landing/images/image_1762966867220.jpg -> /images/image_1762966867220.jpg
Copied: persistent-landing/images/image_1762966867282.jpg -> /images/image_1762966867282.jpg
Copied: persistent-landing/images/image_1762966867347.jpg -> /images/image_1762966867347.jpg
Copied: persistent-landing/images/image_1762966867413.jpg -> /images/image_1762966867413.jpg
Copied: persistent-landing/images/image_1762966867474.jpg -> /images/image_1762966867474.jpg
Copied: persistent-landing/images/image_1762966867538.jpg -> /images/i

In [10]:
# At this moment, all buckets should have the same number of files. Let's check that

# Count valid files in each bucket
for bucket in s3.list_buckets()["Buckets"]:
    
    bucket_name = bucket["Name"]
    count = 0

    paginator = s3.get_paginator("list_objects_v2")
    for page in paginator.paginate(Bucket=bucket_name):
        
        if "Contents" in page:
            for obj in page["Contents"]:
                
                key = obj["Key"]
                
                # Skip folder placeholders (end with "/") and metadata/system files
                if not key.endswith("/") and not key.startswith(("_", ".")):
                    count += 1

    print(f"Bucket: {bucket_name}, Files: {count}")

Bucket: formatted-zone, Files: 996
Bucket: landing-zone, Files: 996
